In [1]:
import numpy as np
import os
import scipy.stats as sps
import scipy.special as spsp
import scipy.linalg as spl
from functools import reduce
import discrete_convolution_statistics as dcs
import pandas as pd

path_proj = os.path.dirname(os.getcwd())+ '/'
path_csv = path_proj + 'csv/'
path_plot = path_proj + 'figures/'

print(path_proj)

#PARAMETERS
#Number of model simulations
n_repeats = 100000
#Level of significance
alpha = 0.05
#Ranks to be tested/degrees of freedom, for the convolution statistics
ranks = np.array([1, 2], dtype=int)
#Set seed for the random number generator
np.random.seed(123)

/Users/quanti/Desktop/Convolution_statistic/Discrete_convolution_statistic/


In [2]:
def bb_dist(n, a, b):
    return np.asarray([spsp.binom(n,k) * spsp.beta(k+a,n-k+b) / spsp.beta(a,b)
                        for k in range(int(n)+1)])

def bb_rho1(n, p):
    res = np.zeros(n+1)
    res[0] = 1.-p
    res[-1] = p
    return res
    
def bb_rho0(n, p):
    return np.asarray([spsp.binom(n, k) * p**(k) * (1.-p)**(n-k)
                        for k in range(n+1)])
            
def BB_pmf(n, p, rho):
    if rho == 0.:
        return bb_rho0(n, p)
    elif rho == 1.:
        return bb_rho1(n, p)
    else:
        a = p * (1.-rho)/rho
        b = (1.-p) * (1.-rho)/rho
        return bb_dist(n, a, b)

def mixBe_pmf(p, q, rho):
    p2 = p * q + np.sqrt(p*(1.-p) * q*(1.-q))
    pmf_1 = np.array([1.-p2, 0., p2])
    pmf_1 = pmf_1/sum(pmf_1)
    return np.convolve(BB_pmf(1, p, 0.), BB_pmf(1, q, 0.)) * (1.-rho) + (rho * pmf_1)

def draw_multinomials(nn, x_lst):
    return np.array([np.random.multinomial(nn[k], x_lst[k]) for k in range(len(x_lst))])

def count_of_sum(cnt_x, support, min_nx):
    if min_nx == 0:
        return np.zeros(len(support))
    else:
        obs = np.zeros(min_nx)
        for k in range(len(cnt_x)):
            temp_obs = np.hstack([np.ones(cnt_x[k,i])*i for i in range(len(cnt_x[k]))])
            obs = obs + np.random.choice(temp_obs, size=min_nx, replace=False)
        obs_support, counts = np.unique(obs, return_counts=True)    
        return np.hstack([counts[obs_support==k] if any(obs_support==k) else np.zeros(1, dtype=int)
                          for k in support])

In [3]:
#Run one simulation of the model X_1 + X_2 and Y, having
#X_1~Be(p) with sample size nn[0], X_2~Be(q) with sample size nn[1],
#and Y~Be(p)+Be(q) with sample size nn[2] where Be(p), Be(q) are correlated via rho.
#Hypotheses of goodness-of-fit (gof) X_1 + X_2 ~ z and equality in distribution (ed) X_1 + X_2 ~ Y
#are tested using these statistics: Pearson's, Convolution, and
#Convolution using true covariance matrix instead of being estimated.
#For the Convolution statistics, the degrees of freedom/covariance matrix in ranks are assayed.
#'run_simulations' output is the proportion of rejections from these statistics,
#approximated via Monte Carlo method repeating the experiment n_repeats times.
#If filename_pval_dist differs from '', then the n_repeats p-values, from each of the statistics above,
#are exported into a csv file, named after filename_pval_dist.
def run_simulations(p, q, rho, ranks, nn, alpha, n_repeats, filename_pval_dist=''):
    
    #Determine true distributions of X_i and Y_i and Z
    lst_prob = [p, q]
    x_lst = [BB_pmf(1, p, 0.) for p in lst_prob]
    y_lst = [mixBe_pmf(p, q, rho)]
    z = reduce(np.convolve, y_lst)
    
    #Minimum samples' size, assumed to be among the X_i for this model
    mm = min(nn)
    
    #Simulations
    counts_x = np.array([draw_multinomials(nn[:len(x_lst)], x_lst) for once in range(n_repeats)])
    counts_y = np.array([draw_multinomials(nn[-len(y_lst):], y_lst) for once in range(n_repeats)])

    #Determine true covariance matrices
    gofSig = dcs.conv_covar(x_lst, mm/nn[:len(x_lst)])
    if len(y_lst) > 1:
        edSig = gofSig + dcs.conv_covar(y_lst, mm/nn[-len(y_lst):])
    else:
        edSig = gofSig + dcs.mult_cov(y_lst[0])*mm/nn[-1]
    
    #Truncate simulations and sum variables into X_1 + X_2
    min_nx = min([cnt_x.sum() for cnt_x in counts_x[0]])
    counts_sum_x = np.array([count_of_sum(cnt_x, np.arange(len(z)), min_nx)
                             for cnt_x in counts_x])
    #There is only one Y variable in this model, so summing is not necessary
    counts_sum_y = np.array([oy[0] for oy in counts_y])

    #Chi2 test (fixed rank)
    gof_chi2 = np.array([dcs.chi2_gof(trc_ob, min_nx*z) for trc_ob in counts_sum_x])
    ed_chi2 = np.array([dcs.chi2_ed(np.array([ox, oy])) for ox,oy in zip(counts_sum_x, counts_sum_y)])

    gof_conv = {}
    ed_conv = {}
    gof_true = {}
    ed_true = {}
    for rk in ranks:
        #Conv test
        gof_conv[rk] = np.array(list(
            map(lambda ox: dcs.conv_test(ary_obsx=ox, gof_z=z,
                                         rk=rk, bool_force_rank=True),
                counts_x)
        ))
        ed_conv[rk] = np.array(list(
            map(lambda ox,oy: dcs.conv_test(ary_obsx=ox, ary_obsy=oy,
                                            rk=rk, bool_force_rank=True),
                counts_x, counts_y)
        ))

        #True sigma conv test
        gof_true[rk] = np.array(list(
            map(lambda ox: dcs.conv_test_true_S(sigma=gofSig, ary_obsx=ox, gof_z=z,
                                                rk=rk, bool_force_rank=True),
                counts_x)
        ))
        ed_true[rk] = np.array(list(
            map(lambda ox,oy: dcs.conv_test_true_S(sigma=edSig, ary_obsx=ox, ary_obsy=oy,
                                                   rk=rk, bool_force_rank=True),
                counts_x, counts_y)
        ))
        
    if filename_pval_dist != '':
        dct_pear = {'Pearson_gof':gof_chi2[:,1], 'Pearson_ed':ed_chi2[:,1]}
        dct_conv_gof = {'Conv_gof_'+str(rk):gof_conv[rk][:,1] for rk in ranks}
        dct_conv_ed = {'Conv_ed_'+str(rk):ed_conv[rk][:,1] for rk in ranks}
        dct_true_gof = {'True_gof_'+str(rk):gof_true[rk][:,1] for rk in ranks}
        dct_true_ed = {'True_ed_'+str(rk):ed_true[rk][:,1] for rk in ranks}
        df = pd.DataFrame({**dct_pear, **dct_conv_gof, **dct_conv_ed, **dct_true_gof, **dct_true_ed})     
        df.to_csv(filename_pval_dist, index=False)
        
    #Proportions of rejection (por)
    por_gof_Pearson = (gof_chi2[:,1] < alpha).sum() / n_repeats
    por_ed_Pearson = (ed_chi2[:,1] < alpha).sum() / n_repeats
    por_gof_conv = np.array([(gof_conv[rk][:,1] < alpha).sum() / n_repeats for rk in ranks])
    por_ed_conv = np.array([(ed_conv[rk][:,1] < alpha).sum() / n_repeats for rk in ranks])
    por_gof_true = np.array([(gof_true[rk][:,1] < alpha).sum() / n_repeats for rk in ranks])
    por_ed_true = np.array([(ed_true[rk][:,1] < alpha).sum() / n_repeats for rk in ranks])
    return np.hstack(((por_gof_Pearson, por_ed_Pearson),
                      por_gof_conv, por_ed_conv, por_gof_true, por_ed_true))

In [4]:
#FIG 1

#PARAMETERS
lst_pq = [(0.3, 0.8), (0.1, 0.9)]
lst_nn = [np.array([10., 10., 10.]), np.array([10., 20., 20.]), np.array([100., 100., 100.])]
lst_rho = np.arange(0., 0.4, 0.01)

#CHECK RULES-OF-THUMB
for p, q in lst_pq:
    for nn in lst_nn:
        m = min(nn[:2])
        ary_x = np.array([BB_pmf(1, k, 0.) for k in [p, q]])
        z = np.array(list(reduce(np.convolve, ary_x)))
        print ('n =', nn, 'p =', p, 'q =', q)
        print ('ALL X_i PASS THE RULE-OF-THUMB:', all(np.ravel(m * ary_x) >= 1))
        print ('Z PASSES THE RULE-OF-THUMB:', all(m * z >= 1), '\n')

#SIMULATIONS
data = []
i = 0
for nb,nn in enumerate(lst_nn):
    for npq,(p,q) in enumerate(lst_pq):
        for nrho,rho in enumerate(lst_rho):
            i += 1
            print (nn, (p,q), rho, i)
            if rho == 0:
                label = path_csv + '_'.join([str(k) for k in [nn, p, q, rho]]) + '.csv'
                row = run_simulations(p, q, rho, ranks, nn, alpha, n_repeats,
                                      filename_pval_dist=label)
            else:
                row = run_simulations(p, q, rho, ranks, nn, alpha, n_repeats)
            data.append(np.hstack([row, nn, [rho, p, q]]))

df = pd.DataFrame(
    data=data, columns=['Pgof', 'Ped']
                       + ['Cgof_'+str(rk) for rk in ranks] + ['Ced_'+str(rk) for rk in ranks]
                       + ['Zgof_'+str(rk) for rk in ranks] + ['Zed_'+str(rk) for rk in ranks]
                       + ['n1', 'n2', 'n3', 'rho', 'X1_p', 'X2_q']
)
df.to_csv(path_csv+'Fig1.csv', index=False)

n = [10. 10. 10.] p = 0.3 q = 0.8
ALL X_i PASS THE RULE-OF-THUMB: True
Z PASSES THE RULE-OF-THUMB: True 

n = [10. 20. 20.] p = 0.3 q = 0.8
ALL X_i PASS THE RULE-OF-THUMB: True
Z PASSES THE RULE-OF-THUMB: True 

n = [100. 100. 100.] p = 0.3 q = 0.8
ALL X_i PASS THE RULE-OF-THUMB: True
Z PASSES THE RULE-OF-THUMB: True 

n = [10. 10. 10.] p = 0.1 q = 0.9
ALL X_i PASS THE RULE-OF-THUMB: False
Z PASSES THE RULE-OF-THUMB: False 

n = [10. 20. 20.] p = 0.1 q = 0.9
ALL X_i PASS THE RULE-OF-THUMB: False
Z PASSES THE RULE-OF-THUMB: False 

n = [100. 100. 100.] p = 0.1 q = 0.9
ALL X_i PASS THE RULE-OF-THUMB: True
Z PASSES THE RULE-OF-THUMB: True 

[10. 10. 10.] (0.3, 0.8) 0.0 1
[10. 10. 10.] (0.3, 0.8) 0.01 2
[10. 10. 10.] (0.3, 0.8) 0.02 3
[10. 10. 10.] (0.3, 0.8) 0.03 4
[10. 10. 10.] (0.3, 0.8) 0.04 5
[10. 10. 10.] (0.3, 0.8) 0.05 6
[10. 10. 10.] (0.3, 0.8) 0.06 7
[10. 10. 10.] (0.3, 0.8) 0.07 8
[10. 10. 10.] (0.3, 0.8) 0.08 9
[10. 10. 10.] (0.3, 0.8) 0.09 10
[10. 10. 10.] (0.3, 0.8) 0.1 11
[1

[100. 100. 100.] (0.1, 0.9) 0.19 220
[100. 100. 100.] (0.1, 0.9) 0.2 221
[100. 100. 100.] (0.1, 0.9) 0.21 222
[100. 100. 100.] (0.1, 0.9) 0.22 223
[100. 100. 100.] (0.1, 0.9) 0.23 224
[100. 100. 100.] (0.1, 0.9) 0.24 225
[100. 100. 100.] (0.1, 0.9) 0.25 226
[100. 100. 100.] (0.1, 0.9) 0.26 227
[100. 100. 100.] (0.1, 0.9) 0.27 228
[100. 100. 100.] (0.1, 0.9) 0.28 229
[100. 100. 100.] (0.1, 0.9) 0.29 230
[100. 100. 100.] (0.1, 0.9) 0.3 231
[100. 100. 100.] (0.1, 0.9) 0.31 232
[100. 100. 100.] (0.1, 0.9) 0.32 233
[100. 100. 100.] (0.1, 0.9) 0.33 234
[100. 100. 100.] (0.1, 0.9) 0.34 235
[100. 100. 100.] (0.1, 0.9) 0.35000000000000003 236
[100. 100. 100.] (0.1, 0.9) 0.36 237
[100. 100. 100.] (0.1, 0.9) 0.37 238
[100. 100. 100.] (0.1, 0.9) 0.38 239
[100. 100. 100.] (0.1, 0.9) 0.39 240


In [5]:
#FIG 2

#PARAMETERS
lst_pq = [(0.3, 0.8), (0.1, 0.9)]
lst_rho = np.array([0., 0.25])
lst_n = np.array([10., 25., 50., 75., 100., 250., 500., 750., 1000.])
base_nn = np.array([[1., 1., 1.], [1., 2., 2.]])

#SIMULATIONS
data = []
i = 0
for npq,(p,q) in enumerate(lst_pq):
    for nrho,rho in enumerate(lst_rho):
        for nb,base in enumerate(base_nn):
            for n in lst_n:
                nn = base * n
                i += 1
                print(nn, (p,q), rho, i)
                row = run_simulations(p, q, rho, ranks, nn, alpha, n_repeats)
                data.append(np.hstack([row, base, [rho, p, q, n]]))

df = pd.DataFrame(
    data=data, columns=['Pgof', 'Ped']
                       + ['Cgof_'+str(rk) for rk in ranks] + ['Ced_'+str(rk) for rk in ranks]
                       + ['Zgof_'+str(rk) for rk in ranks] + ['Zed_'+str(rk) for rk in ranks]
                       + ['n1', 'n2', 'n3', 'rho', 'X1_p', 'X2_q', 'm']
)
df.to_csv(path_csv+'Fig2.csv', index=False)

[10. 10. 10.] (0.3, 0.8) 0.0 1
[25. 25. 25.] (0.3, 0.8) 0.0 2
[50. 50. 50.] (0.3, 0.8) 0.0 3
[75. 75. 75.] (0.3, 0.8) 0.0 4
[100. 100. 100.] (0.3, 0.8) 0.0 5
[250. 250. 250.] (0.3, 0.8) 0.0 6
[500. 500. 500.] (0.3, 0.8) 0.0 7
[750. 750. 750.] (0.3, 0.8) 0.0 8
[1000. 1000. 1000.] (0.3, 0.8) 0.0 9
[10. 20. 20.] (0.3, 0.8) 0.0 10
[25. 50. 50.] (0.3, 0.8) 0.0 11
[ 50. 100. 100.] (0.3, 0.8) 0.0 12
[ 75. 150. 150.] (0.3, 0.8) 0.0 13
[100. 200. 200.] (0.3, 0.8) 0.0 14
[250. 500. 500.] (0.3, 0.8) 0.0 15
[ 500. 1000. 1000.] (0.3, 0.8) 0.0 16
[ 750. 1500. 1500.] (0.3, 0.8) 0.0 17
[1000. 2000. 2000.] (0.3, 0.8) 0.0 18
[10. 10. 10.] (0.3, 0.8) 0.25 19
[25. 25. 25.] (0.3, 0.8) 0.25 20
[50. 50. 50.] (0.3, 0.8) 0.25 21
[75. 75. 75.] (0.3, 0.8) 0.25 22
[100. 100. 100.] (0.3, 0.8) 0.25 23
[250. 250. 250.] (0.3, 0.8) 0.25 24
[500. 500. 500.] (0.3, 0.8) 0.25 25
[750. 750. 750.] (0.3, 0.8) 0.25 26
[1000. 1000. 1000.] (0.3, 0.8) 0.25 27
[10. 20. 20.] (0.3, 0.8) 0.25 28
[25. 50. 50.] (0.3, 0.8) 0.25 29
[ 50

In [6]:
#FIG 3

#PARAMETERS
q = 0.8
lst_nn = [np.array([10., 10., 10.]), np.array([10., 20., 20.]), np.array([100., 100., 100.])]
lst_rho = np.array([0., 0.25])
#(p CLOSE TO q)
lst_p_near = np.arange(0.55, 0.8, 0.01)
#(p FAR FROM q)
lst_p_far = np.arange(0.01, 0.251, 0.01)

#SIMULATIONS
data = []
i = 0
for nb,nn in enumerate(lst_nn):
    for nrho,rho in enumerate(lst_rho):
        for nlp,lst_p in enumerate([lst_p_near, lst_p_far]):
            for npq,p in enumerate(lst_p):
                i += 1
                print(nn, (p,q), rho, i)
                row = run_simulations(p, q, rho, ranks, nn, alpha, n_repeats)
                if nlp == 0:
                    data.append(np.hstack([row, nn, [rho, p, q, 'near']]))
                else:
                    data.append(np.hstack([row, nn, [rho, p, q, 'far']]))

df = pd.DataFrame(
    data=data, columns=['Pgof', 'Ped']
                       + ['Cgof_'+str(rk) for rk in ranks] + ['Ced_'+str(rk) for rk in ranks]
                       + ['Zgof_'+str(rk) for rk in ranks] + ['Zed_'+str(rk) for rk in ranks]
                       + ['n1', 'n2', 'n3', 'rho', 'X1_p', 'X2_q', 'where_p']
)
df.to_csv(path_csv+'Fig3.csv', index=False)

[10. 10. 10.] (0.55, 0.8) 0.0 1
[10. 10. 10.] (0.56, 0.8) 0.0 2
[10. 10. 10.] (0.5700000000000001, 0.8) 0.0 3
[10. 10. 10.] (0.5800000000000001, 0.8) 0.0 4
[10. 10. 10.] (0.5900000000000001, 0.8) 0.0 5
[10. 10. 10.] (0.6000000000000001, 0.8) 0.0 6
[10. 10. 10.] (0.6100000000000001, 0.8) 0.0 7
[10. 10. 10.] (0.6200000000000001, 0.8) 0.0 8
[10. 10. 10.] (0.6300000000000001, 0.8) 0.0 9
[10. 10. 10.] (0.6400000000000001, 0.8) 0.0 10
[10. 10. 10.] (0.6500000000000001, 0.8) 0.0 11
[10. 10. 10.] (0.6600000000000001, 0.8) 0.0 12
[10. 10. 10.] (0.6700000000000002, 0.8) 0.0 13
[10. 10. 10.] (0.6800000000000002, 0.8) 0.0 14
[10. 10. 10.] (0.6900000000000002, 0.8) 0.0 15
[10. 10. 10.] (0.7000000000000002, 0.8) 0.0 16
[10. 10. 10.] (0.7100000000000002, 0.8) 0.0 17
[10. 10. 10.] (0.7200000000000002, 0.8) 0.0 18
[10. 10. 10.] (0.7300000000000002, 0.8) 0.0 19
[10. 10. 10.] (0.7400000000000002, 0.8) 0.0 20
[10. 10. 10.] (0.7500000000000002, 0.8) 0.0 21
[10. 10. 10.] (0.7600000000000002, 0.8) 0.0 22
[10

[10. 20. 20.] (0.19, 0.8) 0.25 194
[10. 20. 20.] (0.2, 0.8) 0.25 195
[10. 20. 20.] (0.21000000000000002, 0.8) 0.25 196
[10. 20. 20.] (0.22, 0.8) 0.25 197
[10. 20. 20.] (0.23, 0.8) 0.25 198
[10. 20. 20.] (0.24000000000000002, 0.8) 0.25 199
[10. 20. 20.] (0.25, 0.8) 0.25 200
[100. 100. 100.] (0.55, 0.8) 0.0 201
[100. 100. 100.] (0.56, 0.8) 0.0 202
[100. 100. 100.] (0.5700000000000001, 0.8) 0.0 203
[100. 100. 100.] (0.5800000000000001, 0.8) 0.0 204
[100. 100. 100.] (0.5900000000000001, 0.8) 0.0 205
[100. 100. 100.] (0.6000000000000001, 0.8) 0.0 206
[100. 100. 100.] (0.6100000000000001, 0.8) 0.0 207
[100. 100. 100.] (0.6200000000000001, 0.8) 0.0 208
[100. 100. 100.] (0.6300000000000001, 0.8) 0.0 209
[100. 100. 100.] (0.6400000000000001, 0.8) 0.0 210
[100. 100. 100.] (0.6500000000000001, 0.8) 0.0 211
[100. 100. 100.] (0.6600000000000001, 0.8) 0.0 212
[100. 100. 100.] (0.6700000000000002, 0.8) 0.0 213
[100. 100. 100.] (0.6800000000000002, 0.8) 0.0 214
[100. 100. 100.] (0.6900000000000002, 0.